# Forecasting Experiment on lorenz Dataset

In [1]:
import os
import torch
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

from engine.solver import Trainer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from engine.logger import Logger
from Utils.io_utils import load_yaml_config, instantiate_from_config
from Models.interpretable_diffusion.model_utils import normalize_to_neg_one_to_one, unnormalize_to_zero_to_one

from sklearn import metrics

In [2]:
# print(f"Data shape: {data.shape}")
# np.save('/data/0shared/liubo/diffusion-gc/Diffusion-TS-main/Experiments/data/solar_nips.npy', data)

In [3]:
class get_dataset(Dataset):
    def __init__(self, data, seq_length,mode,train_split=0.8):
        super(get_dataset, self).__init__()

        self.samples = data
        
        self.seq_length=seq_length
        self.pred_length = 0
        self.features = data.shape[-1]
        
        self.data = self.get_data(data)
        train_num = int(train_split * len(self.data))
        if mode == 'train':
            self.data = self.data[:train_num, :, :]
        else:
            self.data = self.data[train_num:, :, :]
        
    def __getitem__(self, index):
        return self.data[index, :, :]
                

    def __len__(self):
        return len(self.data)
    
    def get_data(self,data):

        # data_max = np.max(data, axis=0)
        # data_min = np.min(data, axis=0)
 
        # data = (data - data_min) / (data_max - data_min)
        num_sample = len(data) - self.seq_length - self.pred_length + 1
        seq_data = torch.zeros(num_sample,
                               self.seq_length + self.pred_length,
                               self.features)
 
        #         print(data.iloc[0:0 + self.seq_length + 1, self.features].values)
 
        for i in range(num_sample):
            seq_data[i] = torch.tensor(data[i:i + self.seq_length + self.pred_length,
                                       :])
        #         print(data_max)
        #         print(data_min)
 
        return seq_data


data=np.load('/data/0shared/liubo/diffusion-gc/DiffuGC/my_exp/lorenz/lorenz_p15_t1000_F20.npy')
print(data.shape)   

train_split=0.8
batch_size=64

train = data[:int(train_split*data.shape[0]), :]
test = data[int(train_split*data.shape[0]):,:].reshape(1, -1, data.shape[-1])

# scaler = MinMaxScaler()
# train_scaled = normalize_to_neg_one_to_one(scaler.fit_transform(train))
# test_scaled = scaler.transform(test.reshape(-1, test.shape[-1])).reshape(test.shape)
# test_scaled = normalize_to_neg_one_to_one(test_scaled)

# print(f"Train shape: {train_scaled.shape}")
# print(f"Test shape: {test_scaled.shape}")

class Args_Example:
    def __init__(self) -> None:
        self.name='lorenz_p15_t1000_f40'
        self.config_path = './Config/lorenz.yaml'
        self.save_dir = '/data/0shared/liubo/diffusion-gc/DiffuGC/my_exp/lorenz_p15_t1000_f40'
        self.gpu = 0
        os.makedirs(self.save_dir, exist_ok=True)

args =  Args_Example()
configs = load_yaml_config(args.config_path)
device = torch.device(f'cuda:{args.gpu}' if torch.cuda.is_available() else 'cpu')
train_dataset = get_dataset(train, seq_length=configs['model']['params']['seq_length'], mode='train', train_split=train_split)
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

log_name= 'seq_length'+str(configs['model']['params']['seq_length'])+'_batch_size'+str(batch_size)+'_lr'+str(configs['solver']['base_lr'])+'_d_model'+str(configs['model']['params']['d_model'])+'_num_layers'+str(configs['model']['params']['n_layer_enc'])+'_num_heads'+str(configs['model']['params']['n_heads'])+'_dropout'+str(configs['model']['params']['attn_pd'])





(1000, 15)


In [ ]:
for name, param in model.named_parameters():
    print(f"Parameter Name: {name}")
    # print(f"Requires Grad: {param.requires_grad}")
    # print(f"Parameter Value: {param}")
    print(f"Shape: {param.shape}")
    print("-" * 50)

In [4]:
feature_size=data.shape[1]
inferred_gc=[]
for ts_target in range(feature_size):
# ts_target=0
    print(f"ts_target: {ts_target}")
    gc_logger = Logger(args=args,name=log_name,ts_target=ts_target)
    model = instantiate_from_config(configs['model']).to(device)
    trainer = Trainer(config=configs, args=args, model=model, dataloader=trainloader,weight_decay=configs['solver']['weight_decay'],logger=gc_logger)
    norm=trainer.train(ts_target)
    inferred_gc.append(norm)

ts_target: 0
{'overall': {'trainable': '17.21M', 'non_trainable': '0', 'total': '17.21M'}, 'denoise_Transformer': {'trainable': '17.21M', 'non_trainable': '0', 'total': '17.21M'}}
lorenz_p15_t1000_f40: start training...


Epoch:50|train loss: 36.3614|l_recon: 16.2293|ridge_loss: 20.1322|l2_loss:11.6587|lr: 0.000500|prox_lam:5.00:   2%|▎         | 50/2000 [01:11<18:46,  1.73it/s]        

lorenz_p15_t1000_f40: train: Epoch 50/2000 Reconstruction Loss: 16.2293 Reglarization: 20.1322 | Total Loss: 36.361430 || Learning Rate: 0.000500Prox lambda: 5.0000start lr: 0.0000 Weight decay: 0.0000d_model: 512.0000 d_ff: 2048.0000 n_heads: 4.0000 n_layer_enc: 2.0000 n_layer_dec: 2.0000 seq_length: 32.0000 Batch size: 64.0000 Dropout: 0.1000
 Norm: [[0.76170695 0.62853914 0.55629945 0.60277957 0.5132062  0.51085454
  0.4822889  0.4876593  0.5616597  0.5596806  0.5003783  0.47243553
  0.6105883  0.52702683 0.56070244]]
Norm: tensor([[0.7617, 0.6285, 0.5563, 0.6028, 0.5132, 0.5109, 0.4823, 0.4877, 0.5617,
         0.5597, 0.5004, 0.4724, 0.6106, 0.5270, 0.5607]], device='cuda:0',
       grad_fn=<LinalgVectorNormBackward0>)


Epoch:100|train loss: 23.0949|l_recon: 13.4610|ridge_loss: 9.6339|l2_loss:9.8948|lr: 0.000500|prox_lam:5.00:   5%|▌         | 100/2000 [01:42<18:52,  1.68it/s]

lorenz_p15_t1000_f40: train: Epoch 100/2000 Reconstruction Loss: 13.4610 Reglarization: 9.6339 | Total Loss: 23.094926 || Learning Rate: 0.000500Prox lambda: 5.0000start lr: 0.0000 Weight decay: 0.0000d_model: 512.0000 d_ff: 2048.0000 n_heads: 4.0000 n_layer_enc: 2.0000 n_layer_dec: 2.0000 seq_length: 32.0000 Batch size: 64.0000 Dropout: 0.1000
 Norm: [[1.4932455e-01 1.2351561e-02 1.8965475e-04 0.0000000e+00 0.0000000e+00
  2.3804093e-04 0.0000000e+00 9.0045924e-04 3.7945923e-04 4.5263651e-04
  8.3191786e-05 1.3359522e-03 1.4326266e-03 1.3474318e-02 1.4512556e-02]]
Norm: tensor([[1.4932e-01, 1.2352e-02, 1.8965e-04, 0.0000e+00, 0.0000e+00, 2.3804e-04,
         0.0000e+00, 9.0046e-04, 3.7946e-04, 4.5264e-04, 8.3192e-05, 1.3360e-03,
         1.4326e-03, 1.3474e-02, 1.4513e-02]], device='cuda:0',
       grad_fn=<LinalgVectorNormBackward0>)


...

lorenz_p15_t1000_f40: train: Epoch 300/2000 Reconstruction Loss: 9.1252 Reglarization: 0.6579 | Total Loss: 9.783065 || Learning Rate: 0.000125Prox lambda: 8.0000start lr: 0.0000 Weight decay: 0.0000d_model: 512.0000 d_ff: 2048.0000 n_heads: 4.0000 n_layer_enc: 2.0000 n_layer_dec: 2.0000 seq_length: 32.0000 Batch size: 64.0000 Dropout: 0.1000
 Norm: [[2.8501963e-05 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 1.2849366e-04 0.0000000e+00 3.9393869e-03]]
Norm: tensor([[2.8502e-05, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.2849e-04, 0.0000e+00, 3.9394e-03]], device='cuda:0',
       grad_fn=<LinalgVectorNormBackward0>)


Epoch:350|train loss: 9.1784|l_recon: 8.6044|ridge_loss: 0.5740|l2_loss:9.4318|lr: 0.000031|prox_lam:9.00:  18%|█▊        | 350/2000 [04:58<18:06,  1.52it/s] 

lorenz_p15_t1000_f40: train: Epoch 350/2000 Reconstruction Loss: 8.6044 Reglarization: 0.5740 | Total Loss: 9.178387 || Learning Rate: 0.000031Prox lambda: 9.0000start lr: 0.0000 Weight decay: 0.0000d_model: 512.0000 d_ff: 2048.0000 n_heads: 4.0000 n_layer_enc: 2.0000 n_layer_dec: 2.0000 seq_length: 32.0000 Batch size: 64.0000 Dropout: 0.1000
 Norm: [[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 6.2245534e-05 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 3.0747312e-03]]
Norm: tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 6.2246e-05, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 3.0747e-03]], device='cuda:0',
       grad_fn=<LinalgVectorNormBackward0>)


Epoch:370|train loss: 8.9492|l_recon: 8.3925|ridge_loss: 0.5567|l2_loss:8.4039|lr: 0.000016|prox_lam:10.00:  18%|█▊        | 370/2000 [05:13<22:59,  1.18it/s]

Early stopping at epoch 370
training complete
最佳训练损失: 7.894440 (Epoch 270)
lam: 9.999999999999982,lr: 1e-05, norm: [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.000127   0.
  0.         0.         0.00545257]]
Training done, time: 313.12


In [33]:
inferrence=[i.cpu().detach().numpy() for i in inferred_gc]
inferrence=np.array(inferrence)
for target in range(feature_size):
    print(f"target: {target}")
    ground=np.load('/data/0shared/liubo/diffusion-gc/DiffuGC/my_exp/lorenz_p15_t1000_f40_GC.npy')
    auprc = metrics.average_precision_score(ground[target,:], inferrence[target,:])
    auroc = metrics.roc_auc_score(ground[target,:], inferrence[target,:])
    print(f"auprc: {auprc}, auroc: {auroc}")

target: 0
auprc: 1.0, auroc: 1.0
target: 1
auprc: 1.0, auroc: 1.0
target: 2
auprc: 1.0, auroc: 1.0
target: 3
auprc: 0.45, auroc: 0.625
target: 4
auprc: 1.0, auroc: 1.0
target: 5
auprc: 1.0, auroc: 1.0
target: 6
auprc: 0.55, auroc: 0.6818181818181817
target: 7
auprc: 0.45, auroc: 0.5909090909090908
target: 8
auprc: 0.48333333333333334, auroc: 0.6136363636363635
target: 9
auprc: 1.0, auroc: 1.0
target: 10
auprc: 0.45, auroc: 0.5909090909090908
target: 11
auprc: 0.45, auroc: 0.625
target: 12
auprc: 0.45, auroc: 0.5909090909090908
target: 13
auprc: 0.45, auroc: 0.5909090909090908
target: 14
auprc: 0.45, auroc: 0.5909090909090908


In [12]:
norm=[1.6527136e-03,3.0629034e-04, 3.0610681e-07 ,0.0000000e+00, 0.0000000e+00,
  0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
  0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 3.1029538e-04, 7.8968407e-04]
ground=[1,1,0,0,0,0,0,0,0,0,0,0,0,1,1]
auprc = metrics.average_precision_score(ground, norm)
print("AUROC:", auprc)
auroc = metrics.roc_auc_score(ground, norm)
print("AUROC:", auroc)

AUROC: 1.0
AUROC: 1.0


In [ ]:
_, seq_length, feat_num = test.shape
pred_length = 24

test_dataset = get_dataset(test, regular=False, pred_length=pred_length)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=test.shape[0], shuffle=False, num_workers=0, pin_memory=True, sampler=None)

sample, *_ = trainer.restore(test_dataloader, shape=[seq_length, feat_num], coef=1e-1, stepsize=5e-2, sampling_steps=200)
mask = test_dataset.mask
mse = mean_squared_error(sample[~mask], real[~mask])
print(mse)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["font.size"] = 12

for idx in range(5):
    plt.figure(figsize=(15, 3))
    plt.plot(range(0, seq_length-pred_length), real[0, :(seq_length-pred_length), -idx], color='c', linestyle='solid', label='History')
    plt.plot(range(seq_length-pred_length-1, seq_length), real[0, -pred_length-1:, -idx], color='g', linestyle='solid', label='Ground Truth')
    plt.plot(range(seq_length-pred_length-1, seq_length), sample[0, -pred_length-1:, -idx], color='r', linestyle='solid', label='Prediction')
    plt.tick_params('both', labelsize=15)
    plt.subplots_adjust(bottom=0.1, left=0.05, right=0.99, top=0.95)
    plt.legend()
    plt.show()